# Storm Events - Cost and Fatalities
### The following notebook works with a dataset involving major storm events, the damage associated with them, and the loss of life involved in these events. The goal here is to show you how these datasets can be characterized, worked with, and interpreted. 

In [ ]:
#Import our necessary packages
import numpy as np
import pandas as pd
from pandas import Series, DataFrame
from scipy import stats
import matplotlib.pyplot as plt
from datetime import datetime

In [ ]:
#look at the raw data files. For explanations, see Storm-Data-Export-Format.pdf in the data directory
sed = pd.read_csv('atmos_5040_2019/data/StormEvents_details_2018.csv',delimiter=',')
sef = pd.read_csv('atmos_5040_2019/data/StormEvents_fatalities_2018.csv',delimiter=',')

In [ ]:
#show event details- note you don't see all columns and rows
sed

In [ ]:
# determine the number of events 
no_sed = len(sed)
no_sed

In [ ]:
#what are the column labels
sed.columns

In [ ]:
#what types of events have happened in 2018?
types_events = sed.EVENT_TYPE.unique()
types_events

In [ ]:
#just trust me, this converts 1.00K to 1000, 1.50M to 1500000, etc.
sed.DAMAGE_PROPERTY = (sed.DAMAGE_PROPERTY.replace(r'[KMB]+$', '', regex=True).astype(float) * \
sed.DAMAGE_PROPERTY.str.extract(r'[\d\.]+([KMB]+)', expand=False).fillna(1).replace(['K','M','B'], [10**3, 10**6, 10**9]).astype(int))

In [ ]:
sed.DAMAGE_CROPS = (sed.DAMAGE_CROPS.replace(r'[KMB]+$', '', regex=True).astype(float) * \
sed.DAMAGE_CROPS.str.extract(r'[\d\.]+([KMB]+)', expand=False).fillna(1).replace(['K','M','B'], [10**3, 10**6, 10**9]).astype(int))

In [ ]:
#total $ losses
total_losses=sed.DAMAGE_PROPERTY.sum()+sed.DAMAGE_CROPS.sum()
print("$",total_losses)
# yowsa!

In [ ]:
sed['TOTAL_LOSS'] = sed.apply(lambda row: row['DAMAGE_PROPERTY']+row['DAMAGE_CROPS'],axis=1)
sed['OVER_MIL_COUNT'] = sed.apply(lambda row: 1 if row['TOTAL_LOSS']>1000000 else 0,axis=1)
sed['OVER_MIL_VAL'] = sed.apply(lambda row: row['TOTAL_LOSS'] if row['TOTAL_LOSS']>1000000 else 0,axis=1)

events = pd.DataFrame(sed.groupby('EVENT_TYPE').size(),columns = ['NUMBER'])
events['COST'] = sed.groupby('EVENT_TYPE').sum()[['DAMAGE_PROPERTY','DAMAGE_CROPS']].apply(lambda row: row['DAMAGE_PROPERTY']+row['DAMAGE_CROPS'],axis=1)
events['NUMBER_MIL'] = sed.groupby('EVENT_TYPE').sum()[['OVER_MIL_COUNT']]
events['COST_MIL'] = sed.groupby('EVENT_TYPE').sum()[['OVER_MIL_VAL']]
events

### Question 1. Do you think the cost of droughts during 2018 is really zero? What other categories might be subject to lack of reporting and why? What surprises you in terms of the types of events, how many occurred, and how much they cost?

#### Question 1 Answer Here

In [ ]:
#what are the costliest types of events in 2018?
ev_cost_large=events.nlargest(10, 'COST')
ev_cost_large

In [ ]:
#crude pie plot of the costliest event categories
fig,ax=plt.subplots(figsize=(5,10))
plt.pie(ev_cost_large['COST'])
plt.legend(ev_cost_large.index,bbox_to_anchor=(0.9, 0.8))
plt.show()

In [ ]:
#show fatality details
sef

In [ ]:
#fatality column headers
print(sef.columns)

In [ ]:
#number of fatalities
no_sef = len(sef)
print(no_sef)

In [ ]:
#what types of locations have been where fatalities occurred?
types_locations = sef.FATALITY_LOCATION.unique()
print(types_locations)

In [ ]:
#how many of these types of locations have happened and how many fatalities have occurrred for each?
#create new dataframe to hold summary statistics
fatalities = pd.DataFrame(sef.groupby('FATALITY_LOCATION').size(),columns = ['NUMBER'])
fatalities

### Question 2. Create a pie plot of the fatalities by location
Hint: Look at the code above!

In [ ]:
#Question 2 code here


### Question 3. Use one line plot a histogram of the ages of the fatalities
hint: dataframe.hist(column='...') 

In [ ]:
#Question 3 code here

### Followup Question 3.1- is the risk of dying higher for the young (under 30) or old (over 60) from a weather event?

Question 3.1 Answer

In [ ]:
#combine the year,month,day into one variable
sef['DATE']=pd.to_datetime(sef.FAT_YEARMONTH*100 + sef.FAT_DAY,format='%Y%m%d')

In [ ]:
#what days of the week did the fatalities happen?
sef['DOW']=sef.DATE.dt.day_name()
sef['DOW_num'] = sef.DATE.dt.dayofweek

In [ ]:
#Create a series object wiht the count of how many fatalities occured on each day of the week
sef_dow_counts = sef.groupby('DOW').size()
sef_dow_counts

### Question 4. Plot a histogram of the days of the week fatalities occurred
(Answer Given)

In [ ]:
# here's how not to do it
sef.hist(column='DOW_num')
plt.show()

In [ ]:
sef['DOW'].value_counts().plot(kind='bar');

In [ ]:
#Try doing it this way:
fig,ax = plt.subplots(figsize=(10,5))
labels = sef_dow_counts.index.values
labels_pos = np.arange(len(labels))
plt.bar(labels_pos,sef_dow_counts)
plt.xticks(labels_pos,labels)
plt.show()

In [ ]:
#group all by the date and then count how many are in each day
sef_date_count = sef.groupby(sef['DATE']).size()
sef_date_count.plot(figsize=(15,4),title='2018 Fatalities')
plt.show()

### Question 5. Write code to output the day within the dataset with the maximum number of fatalities
Hint: use idxmax

In [ ]:
#Question 5 Code here


### Question 6. Use the sef dataframe to print the locations of fatalities that happened on the day with the maximum fatalities.
hint - use "dataframe.loc[somecondition]"     
hint - https://stackoverflow.com/questions/44890713/selection-with-loc-in-python     
hint - https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.loc.html

In [ ]:
#Question 6 Code Here


### Followup question 6.1 - why is so much info unknown/unavailable on this day?

6.1 Answer here

In [ ]:
#now process the events
sed['DATE']=pd.to_datetime(sed.BEGIN_YEARMONTH*100 + sed.BEGIN_DAY,format='%Y%m%d')

### Question 7. Use the "sed" dataframe to print the types of events happened on this day.
hint: use your knowlege of dataframe.loc[somecondition] from question 6 and apply to the "sed" dataframe

In [ ]:
#Question 7 Code Here


### Are you getting a sense of what was the key weather event on max_day_fat?

### Question 8. Create another column in "sed" to store the day of the week that each event happened
hint: sed['DoW'] = refer to between question 3 and 4


In [ ]:
#Question 8 Code Here


### Followup code 8.1. Create a variable called sed_day_count has "DOW" as the index and a count of the number of events that happened on that day of the week as the only column. 
Hint: use groupby    
Hint: look at the code just before question 4


In [ ]:
#Question 8.1 code here

### Question 9. Plot a histogram of the number of events that occurred during each day of the week


In [ ]:
#Question 9 code here

### Question 10. Plot a time series of the events on each day during 2018 - day on the x axis and number of events on the y axis
Hint: Look at just before question 5

In [ ]:
#Question 10 Code

### Question 11. Like question 6, print the day with the maximum number of events. 


In [ ]:
# Question 11 Code


### Followup Question 11.1: Print the events that happened on the day with the max events.


In [ ]:
#Question 11.1 Code

### Question 12. What is the percentage of events that occured in Utah?
Hint: use the "state" column to count the number of events that occured in Utah,  and divide that by the total number of events

In [ ]:
#Question 12 Code

### Question 13. There's a lot of data in these two files. Poke around and come up with something interesting to discuss! This question doesn't require you to write complicated code

In [ ]:
#Question 13 Answer